In [57]:
import sys
sys.path.insert(0, '../../')

In [58]:
from scripts.utils.connect import get_connection 

connection = get_connection()
cursor = connection.cursor()

In [59]:
lookup = [line.strip() for line in open('../../data/moyo/relation.2M.list', 'r')]
print(lookup[:5])

['/people/person/gender', '/music/release/track', '/soccer/football_player/position_s', '/organization/organization_founder/organizations_founded', '/cvg/computer_videogame/cvg_genre']


In [60]:
results = [line.strip().split('\t') for line in open('../../data/moyo/test.replace_ne.withpool', 'r')]
print(results[:5])
print(len(results))

[['40', '61 40 117', 'which genre of album is #head_entity# ?'], ['61', '56 702 132 61 117 40 11', 'what format is #head_entity#'], ['272', '7 1 18 272 308', 'what film is by the writer #head_entity# ?'], ['37', '11 135 1 6 37 18', 'where did #head_entity# die'], ['125', '1 37 18 353 125 162 11 7 1611 414 6 373 537 500', 'what was the cause of death of #head_entity#']]
20609


In [142]:
sum([r[0] == r[1].split()[-1] for r in results]) / len(results)


IndexError: list index out of range

In [64]:
data = [line.strip().split('\t') for line in open('../../data/SimpleQuestions_v2/annotated_fb_data_test.txt', 'r')]
true = [(row[1].replace('www.freebase.com', ''), row[3].lower()) for row in data]
print(true[:5])
print(len(true))

[('/music/album/genre', 'which genre of album is harder.....faster?'), ('/music/album/release_type', 'what format is fearless'), ('/people/person/place_of_birth', 'what city was alex golfis born in'), ('/film/writer/film', 'what film is by the writer phil hay?'), ('/people/deceased_person/place_of_death', 'where did roger marquis die')]
21687


In [25]:
RELATION_CLASSIFIER = '../../logs/0513.12-22_11:09:07.relation_classifier/12m_22d_11h_17m_36s.pt'

import pprint

from nltk.tokenize.treebank import TreebankWordTokenizer

from lib.checkpoint import Checkpoint

pretty_printer = pprint.PrettyPrinter(indent=2)
tokenizer = TreebankWordTokenizer() # Same tokenizer used during training

relation_classifier_predict = Checkpoint(checkpoint_path=RELATION_CLASSIFIER, device=0).predict

def get_relation(question, top_k=20):
    """ 
    Given a question return the predicate in the question using `RELATION_CLASSIFIER` model.
    
    Args:
        question (str)
    Returns:
        list of predicates and their confidence
    """
    question = ' '.join(tokenizer.tokenize(question))
    question = question.replace('# head_entity #', '<e>').lower()
    predicted = list(relation_classifier_predict(question, top_k=top_k))
    return {class_.replace('www.freebase.com/', ''): confidence[0] for class_, confidence in predicted}

# To test this cell
print(get_relation('Where was #head_entity# born?'))

{'people/person/place_of_birth': 0.9966886154947544, 'fictional_universe/fictional_character/place_of_birth': 0.001090397720005712, 'people/person/nationality': 0.0008807870531498447, 'location/location/people_born_here': 0.0007503812682593278, 'people/deceased_person/place_of_death': 0.0005064779033180465, 'music/artist/origin': 6.74701948432796e-05, 'common/topic/notable_types': 3.931290793336356e-06, 'music/recording/releases': 2.1286992065599967e-06, 'people/person/gender': 8.928303538257806e-07, 'film/film/country': 8.883035368870046e-07, 'media_common/netflix_title/netflix_genres': 7.119961528444273e-07, 'people/person/profession': 5.683700328440841e-07, 'music/release/track': 4.4660402933205215e-07, 'music/release/track_list': 4.046538133458398e-07, 'fictional_universe/fictional_setting/fictional_characters_born_here': 3.765283221321857e-07, 'music/artist/track': 3.2377583868610605e-07, 'music/release/format': 2.487110566012019e-07, 'people/profession/people_with_this_profession

In [26]:
import random
my_predictions = []
for res in results:
    question = res[2]
    pool = res[1].split()
    relations = [lookup[int(i) - 1][1:] for i in pool]
    predictions = get_relation(question, 1687)
    max_confidence = 0
    max_relation_index = random.randint(0, len(pool) - 1)
    for i, relation in enumerate(relations):
        if relation in predictions:
            confidence = predictions[relation]
            if confidence > max_confidence:
                max_confidence = confidence
                max_relation_index = i
    if max_confidence == 0:
        print(relations)
        print(question)
        print()
    my_predictions.append('/' + relations[max_relation_index])
my_predictions[:5]

['/symbols/namesake/named_after',
 '/people/person/place_of_birth',
 '/music/release/region',
 '/film/director/film',
 '/music/release/region']

In [89]:
predictions = [(lookup[int(result[0]) - 1], result[2]) for result in results]
print(predictions[:5])

[('/music/album/genre', 'which genre of album is #head_entity# ?'), ('/music/album/release_type', 'what format is #head_entity#'), ('/film/writer/film', 'what film is by the writer #head_entity# ?'), ('/people/deceased_person/place_of_death', 'where did #head_entity# die'), ('/people/deceased_person/cause_of_death', 'what was the cause of death of #head_entity#')]


In [ ]:
# Use the name of the entity predicted for masking...
# TODO: Check if the questions I miss, are the same ones they exclude
# TODO: Do this all in a dataframe such that we can have a clean table with linking similar to my end to end stuff
found = 0
for i, line in enumerate(lines):
    split = line.split('\t')
    if len(split) > 1:
        predicted_label = split[1].split()[0][2:]
        cursor.execute('SELECT alias FROM fb_name WHERE mid = %s', (predicted_label, ))
        rows = cursor.fetchall()
        question = true[i][1]
        for (alias,) in rows:
            if alias in question:
                found += 1
                break;
print('accuracy', found / len(true))
print('found', found)

In [137]:
from nltk.tokenize.stanford import CoreNLPTokenizer
from tqdm import tqdm_notebook

import re
from Levenshtein import distance

tokenizer = CoreNLPTokenizer()
properties = {'tokenize.options': 'ptb3Escaping=true'}

correct = 0
my_correct = 0
my_prediction_index = 0
predictions_index = 0
since_prediction = 0
for true_relation, true_question in tqdm_notebook(true):
    prediction_relation, preprocessed_question = predictions[predictions_index]
    
    true_question = ' '.join(tokenizer.tokenize(true_question, properties=properties)).lower()
    split = preprocessed_question.split('#head_entity#')

    diff = distance(split[0].strip(), true_question) - len(true_question) + len(split[0].strip())
    diff1 = distance(split[1].strip(), true_question) - len(true_question) + len(split[1].strip())
    
    # Differences in `` and ''
    # Differences in / tokenization
    if since_prediction > 10:
        break;
            
    if diff1 + diff < 2:
        since_prediction = 0
#         if my_predictions[my_prediction_index] == true_relation:
#             my_correct += 1
            
        if true_relation == prediction_relation:
            correct += 1
#         else:
#             print(question)
#             print(true_question)
#             print(true_relation)
#             print(prediction_relation)
#             print()
            
        if len(predictions) == predictions_index + 1:
            print('Matched all!')
            break;
            
        predictions_index += 1
        my_prediction_index += 1

    else:
        since_prediction += 1
        print(preprocessed_question)
        print(true_question)
        print(split)
        print(diff1 + diff)
        print()
         
print('accuracy %f [%d of %d]' % (correct / len(true), correct, len(true)))
print('precision %f [%d of %d]' % (correct / len(predictions), correct, len(predictions)))
print('recall %f [%d of %d]' % (len(predictions) / len(true), len(predictions), len(true)))
print('my accuracy', my_correct / len(true))

HBox(children=(IntProgress(value=0, max=21687), HTML(value='')))

what film is by the writer #head_entity# ?
what city was alex golfis born in
['what film is by the writer ', ' ?']
15

who published #head_entity#
who is the creator of loud
['who published ', '']
7

which event took place in #head_entity# -lrb- roman empire -rrb-
who wrote the lyrics for the house that never was
['which event took place in ', ' -lrb- roman empire -rrb-']
29

what is #head_entity# 's profession ?
what genre of album is magico : carta de amor ?
['what is ', " 's profession ?"]
6

what is an album by #head_entity# ?
what kind of film is días de combate ?
['what is an album by ', ' ?']
8

who is the program creator of the show #head_entity# ?
who was an influential figure for miško šuvaković
['who is the program creator of the show ', ' ?']
25

what kind of film is #head_entity#
what gender is daniela cristofori
['what kind of film is ', '']
6

what is a short-lived british #head_entity# series
what is the main ingredient in diamox sequels
['what is a short-lived british 

what is the genre of the book #head_entity#
what kind of music is featured on the album wild . club ` in
['what is the genre of the book ', '']
10

which fictional character was based on #head_entity#
what is eric gosselin 's nationality ?
['which fictional character was based on ', '']
32

who was the executive producer of #head_entity#
what is afonso álvares 's profession ?
['who was the executive producer of ', '']
24

what kind of game is #head_entity# ?
what is curtis graham 's profession ?
['what kind of game is ', ' ?']
9

what artist is in the #head_entity# genre ?
name a sea to go scuba diving in
['what artist is in the ', ' genre ?']
17

what type of school is #head_entity#
what is iqsdirectory.com
['what type of school is ', '']
15

what tv actor was born in #head_entity# ?
what is the title track on another day
['what tv actor was born in ', ' ?']
13

what is the track list off the release #head_entity#
what is language is the composition when in rome in ?
['what is the tra

what is #head_entity# 's nationality ?
what nation is prashant john originally from ?
['what is ', " 's nationality ?"]
4

what film was produced by #head_entity#
what area was affected by the may 18 -- 21 , 2013 tornado outbreak
['what film was produced by ', '']
10

what film was produced by #head_entity#
who directed the movie
['what film was produced by ', '']
23

name a 1971 #head_entity#
what is the legal status of standardized mite d. farinae ?
['name a 1971 ', '']
5

what body of water surrounds #head_entity#
what movie genre is noah considered
['what body of water surrounds ', '']
18

what country is #head_entity# located ?
what is the active ingredient of juvisync
['what country is ', ' located ?']
8

what album was created by #head_entity#
what genre is runways & bunkbeds
['what album was created by ', '']
15

where is #head_entity#
what is an alum released by sizzla
['where is ', '']
2

#head_entity# is by who
what country is campo de ourique in
['', ' is by who']
4

in whi

#head_entity# is the subject of which written work ?
what is famous last words
['', ' is the subject of which written work ?']
41

which song was part of #head_entity#
what is zoi voudouri country of nationality ?
['which song was part of ', '']
12

who is a character that appears in the fictional universe of #head_entity#
which language is perineum millennium - the in-between years written in ?
['who is a character that appears in the fictional universe of ', '']
41

who is the musician that created #head_entity#
who composed bien o mal
['who is the musician that created ', '']
32

what genre does #head_entity# branch off of
what caused the death of brett heinicke ?
['what genre does ', ' branch off of']
10

which new york city did #head_entity# in
what form is single bed in
['which new york city did ', ' in']
17

what artist plays #head_entity#
what gender is aleksandr pipa ?
['what artist plays ', '']
6

what is the profession of #head_entity# ?
which city was johann von vásáry born

...

who is a czech #head_entity#
which country 's borders does pârâul cetăţii fall in
['who is a czech ', '']
5

what format was #head_entity# released as
who wrote the composition i grant you
['what format was ', ' released as']
12

what type of rugby team is #head_entity# in ?
where was john w.g. harley born ?
['what type of rugby team is ', ' in ?']
19

what physical medium was #head_entity# released on
which language is the composition yokohama sing a song -lrb- opening version -rrb- in ?
['what physical medium was ', ' released on']
11

who was #head_entity# named after
what type of music does kazik na żywo perform ?
['who was ', ' named after']
6

what type of film is #head_entity#
what is the legal status of hytrin
['what type of film is ', '']
9

which country is #head_entity# released in ?
where was simon regan born
['which country is ', ' released in ?']
16

what kind of netflix film is #head_entity# ?
which series is the episode jennifer hudson on
['what kind of netflix film is 

where in estonia was #head_entity# born ?
what
['where in estonia was ', ' born ?']
41

who directed the film #head_entity#
what is the gender of stan yale ?
['who directed the film ', '']
11

what 's a song from the album #head_entity#
what is the song from the composition blue
["what 's a song from the album ", '']
5

what play did #head_entity# write lyrics for ?
what is an active ingredient in throat complex
['what play did ', ' write lyrics for ?']
12

what position does #head_entity# play in baseball
who composed ashtabula
['what position does ', ' play in baseball']
25

what is #head_entity# 's job ?
what is the birthplace of joël vanhoebrouck
['what is ', " 's job ?"]
3

whats the name of an album by #head_entity#
what 's a live album by dio ?
['whats the name of an album by ', '']
18

what is the subject of the book `` #head_entity# ?
which musical instrument is played by sebastienfansite
['what is the subject of the book `` ', ' ?']
22

what notable type is the #head_entity# 

what song did #head_entity# write the lyrics for ?
what is a drug under the uniretic class
['what song did ', ' write the lyrics for ?']
14

where did #head_entity# die
who produced zulu ?
['where did ', ' die']
5

what does #head_entity# flow out to ?
which country was the movie released in
['what does ', ' flow out to ?']
6

what genre within blues music does #head_entity# play
what is the genre that joyce bryant falls into ?
['what genre within blues music does ', ' play']
19

what character is a #head_entity# ?
what is the genre of son epouse
['what character is a ', ' ?']
10

what city did #head_entity# die in
who wrote \ \ `` master harold \ \ '' ... and the boys ?
['what city did ', ' die in']
8

what kind of release did #head_entity# have ?
name a drama film
['what kind of release did ', ' have ?']
29

what genre of book is #head_entity#
where was razame de la crackers born
['what genre of book is ', '']
11

what is #head_entity# 's ethnicity
where did baxter1111 start out
['wh

what planet was discovered at the #head_entity# ?
what is the active ingredient in natrabio the arnica rub
['what planet was discovered at the ', ' ?']
18

is scooby doo or antologia an #head_entity#
what format was the goa mix released in ?
['is scooby doo or antologia an ', '']
21

what is the cause of death of #head_entity# ?
which artist released the album
['what is the cause of death of ', ' ?']
22

what is the cause of death of #head_entity# ?
which artist recorded hollywood ?
['what is the cause of death of ', ' ?']
20

who played with #head_entity#
what gender is alice szellay
['who played with ', '']
9

what is #head_entity# 's country of nationality ?
who is the producing company of les misérables
['what is ', " 's country of nationality ?"]
18

what format was #head_entity# released in
who wrote the song graveyard shift
['what format was ', ' released in']
10

what album is the song #head_entity# on
what kind of place of worship is the et ` hem bey mosque
['what album is the

in what conference were the #head_entity#
what 's the active component of the drug brimonidine tartrate
['in what conference were the ', '']
11

in which city did #head_entity# die ?
what song is maybe it 's you a recorded version of ?
['in which city did ', ' die ?']
10

what platform is the game #head_entity# played on
which gender is mack bolan
['what platform is the game ', ' played on']
22

what is a book by #head_entity#
who composed perfect blue buildings
['what is a book by ', '']
9

what battle did #head_entity# participate in
which skin diseases are caused by a fungus ?
['what battle did ', ' participate in']
16

what river does the #head_entity# corss
what city and state is delaware park -- front park system in
['what river does the ', ' corss']
6

what video game platform was #head_entity# released on ?
what 's an ingredient present in glumetza
['what video game platform was ', ' released on ?']
21

who is the singer of #head_entity# album
what is sylvia poggiolis place of 

In [30]:
print(len(predictions) / len(true))

0.9503918856615952


In [7]:
path = '/home/michael/Downloads/valid.fuzzy_p2_linker.simple_linker.original.union'
lines = [line for line in open(path)]
print(len(lines))

10845


In [11]:
correct = 0
for line in lines:
    split = line.split('\t')
    true_label = split[0]
    if len(split) > 1:
        predicted_label = split[1].split()[0]
    else:
        continue
    if true_label == predicted_label:
        correct += 1
print('accuracy', correct / len(true))
print('correct', correct)

accuracy 0.7359151682803136
correct 7981


accuracy 0.9584140156754265
found 10394
